<a href="https://colab.research.google.com/github/ineeddspelchek/Gaza-MisDisinfo-Recognition/blob/main/CS_4710_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install PyICU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyICU: filename=PyICU-2.13.1-cp310-cp310-linux_x86_64.whl size=1801404 sha256=058f7137ba03fb965745fbc71ee02a4be5a1058b2442ebdcecef12ba465fab45
  Stored in directory: /root/.cache/pip/wheels/8e/45/08/f4bc505e31eb7bb779d493226921abf18cf55ed30142e70eae
Successfully built PyICU


Preprocessing

In [ ]:
# Preprocessing/Data - Henry
'''
Sources:
https://stackoverflow.com/questions/48376580/how-to-read-data-in-google-colab-from-my-google-drive
https://stackoverflow.com/questions/62637553/how-to-get-all-unicode-characters-of-a-language-by-using-its-iso-language-code-i
https://www.freecodecamp.org/news/how-to-remove-a-specific-character-from-a-string-in-python/
https://www.w3schools.com/python/
https://stackoverflow.com/questions/33803306/split-strings-on-whitespaces-but-do-not-remove-them
https://www.rexegg.com/regex-quickstart.html
https://stackoverflow.com/questions/12453580/how-to-concatenate-join-items-in-a-list-to-a-single-string
https://skills.ai/blog/import-google-sheets-to-pandas/
https://stackoverflow.com/questions/399078/what-special-characters-must-be-escaped-in-regular-expressions
https://stackoverflow.com/questions/4172448/is-it-possible-to-break-a-long-line-to-multiple-lines-in-python
'''

import pandas as pd
import numpy as np
import random
import icu
import re

googleSheetURL = "https://docs.google.com/spreadsheets/d/10gKGxnbHhRqv0PgDucE36ks93yjSHLgN/export?format=xlsx"

IMPORTED_DATA = pd.read_excel(googleSheetURL, sheet_name="Fact Checks", header=None).to_numpy().tolist()
np.random.shuffle(IMPORTED_DATA)

HEBREW_CHARACTERS = icu.LocaleData('he').getExemplarSet(0, 0)
ARABIC_CHARACTERS = icu.LocaleData('ar').getExemplarSet(0, 0)

ISRAEL_IDENTIFIERS = pd.read_excel(googleSheetURL, sheet_name="Swap Chart", header=None).to_numpy().transpose()[0].tolist()
PALESTINE_IDENTIFIERS = pd.read_excel(googleSheetURL, sheet_name="Swap Chart", header=None).to_numpy().transpose()[1].tolist()

SPLIT_DELIMS = "([" + \
"\"\-#\s.,;()!?" + \
"😀😃😄😁😆🥹😅😂🤣😍😌😉🙃🙂😇😊☺️🥲🥰😘😗😙😚😋😛😝😜😏🥳🤩🥸😎🤓🧐🤨🤪😒😞😔😟😕🙁☹️😣😖🤬😡😠😤😭😢🥺😩😫🤯😳🥵🥶😶‍🌫️😱😨😰😥🫠🤫🫡🫢🤭🫣🤔🤗😓🤥😶🫥😐🫤😑😬🙄😯😮‍💨😪🤤😴🥱😲😮😧😦😵😵‍💫🤐🥴🤢🤮🤧😷🤒💩🤡👺👹👿😈🤠🤑🤕👻💀☠️👽👾🤖🎃😺😸🤲🫶😾😿🙀😽😼😻😹👐🙌👏🤝👍👎✊🤛👊🤏🤌👌🤘🤟🫰✌️🤞🤜🫳🫴👈👉👆👇☝️✋🤚🖕🦾💪🫱🫲🤙👋🖖🖐✍️🙏🫵🦶🦵🦿💄💋👄👀👁👣👃🦻👂👅🦷🫦🫀🫁🧠🗣👤👥🫂👶👧👩‍🦰👨‍🦱🧑‍🦱👩‍🦱👨🧑👩👦🧒🧑‍🦰👨‍🦰👱‍♀️👱👱‍♂️👩‍🦳🧑‍🦳👨‍🦳👩‍🦲👲👴🧓👵🧔‍♂️🧔🧔‍♀️👨‍🦲🧑‍🦲👳‍♀️👳👳‍♂️🧕👮‍♀️👮👮‍♂️👷‍♀️👷🧑‍⚕️👩‍⚕️🕵️‍♂️🕵️🕵️‍♀️💂‍♂️💂💂‍♀️👷‍♂️👨‍⚕️👩‍🌾🧑‍🌾👩‍🍳🧑‍🍳👨‍🍳👩‍🎓🧑‍🎓👨‍🌾🧑‍🏭👩‍🏭👨‍🏫🧑‍🏫👩‍🏫👨‍🎤🧑‍🎤👩‍🎤👨‍🎓👨‍🏭👩‍💻🧑‍💻👨‍💻👩‍💼🧑‍💼👨‍💼👩‍🔧🧑‍🔧🧑‍🚒👩‍🚒👨‍🎨🧑‍🎨👩‍🎨👨‍🔬🧑‍🔬👩‍🔬👨‍🔧👨‍🚒👩‍✈️🧑‍✈️👨‍✈️👩‍🚀🧑‍🚀👨‍🚀👩‍⚖️🧑‍⚖️🫅👸🤵‍♂️🤵🤵‍♀️👰‍♂️👰👰‍♀️👨‍⚖️🤴🥷🦸‍♀️🦸🦸‍♂️🦹🦹‍♂️🤶🦹‍♀️🧌🧝‍♂️🧝🧝‍♀️🧙‍♂️🧙🧙‍♀️🎅🧑‍🎄🧛‍♀️🧛🧛‍♂️🧟‍♀️🧟🧟‍♂️🧞‍♀️🧞🧞‍♂️🫄🤰👼🧚‍♂️🧚🧚‍♀️🧜‍♂️🧜🧜‍♀️🫃🤱👩‍🍼🧑‍🍼👨‍🍼🙇‍♀️🙇🙇‍♂️💁‍♀️🙋‍♀️🙆‍♂️🙆🙆‍♀️🙅‍♂️🙅🙅‍♀️💁‍♂️💁🙋🙋‍♂️🧏‍♀️🧏🧏‍♂️🤦‍♀️🤦🤦‍♂️🤷‍♀️💇‍♀️🙍‍♂️🙍🙍‍♀️🙎‍♂️🙎🙎‍♀️🤷‍♂️🤷💇💇‍♂️💆‍♀️💆💆‍♂️🧖‍♀️🧖🧖‍♂️💅🧑‍🦽👩‍🦽🕴👯‍♂️👯👯‍♀️🕺💃🤳👨‍🦽🧑‍🦼👨‍🦼🚶‍♀️🚶🚶‍♂️👩‍🦯🧑‍🦯🧍🧍‍♀️🏃‍♂️🏃🏃‍♀️🧎‍♂️🧎🧎‍♀️👨‍🦯🧍‍♂️👫👭👬👩‍❤️‍👨👩‍❤️‍👩💑👨‍❤️‍👨👩‍❤️‍💋‍👨👩‍👩‍👦👨‍👩‍👧‍👧👨‍👩‍👦‍👦👨‍👩‍👧‍👦👨‍👩‍👧👨‍👩‍👦👨‍❤️‍💋‍👨💏👩‍❤️‍💋‍👩👩‍👩‍👧👩‍👩‍👧‍👦👩‍👩‍👦‍👦👩‍👩‍👧‍👧👨‍👨‍👦👨‍👨‍👧👨‍👨‍👧‍👦👨‍👨‍👦‍👦👨‍👨‍👧‍👧👨‍👦‍👦👨‍👧‍👦👨‍👧👨‍👦👩‍👧‍👧👩‍👦‍👦👩‍👧‍👦👩‍👧👩‍👦👨‍👧‍👧🪢🧶🧵🪡🧥🥼🦺👚👘🩱👙👗👔🩳🩲👖👕🥻🩴🥿👠👡👢👞👟🥾🪖⛑👒🎓🧢🎩🧣🧤🧦👑💍👝👛👜💼🎒🧳👓🌂🥽🕶❤️🧡💛💚💙💜🤎🖤🤍💖💗💓💞💕❣️❤️‍🩹❤️‍🔥💔💘💝♡♥︎❥❦❧☙❢এლდღɞʚෆᰔᩚᰔঞଓᜊᥫ᭡ꨄஐᦗ🐶🐱🐭🐹🐰🦊🐻🐼🐻‍❄️🙈🐵🐸🐽🐷🐮🦁🐯🐨🙉🙊🐒🐔🐧🐦🐤🐣🐥🐝🦄🐴🐗🐺🦇🦉🦅🦆🪱🐛🦋🐌🐞🐜🪰🪲🪳🦖🦎🐍🐢🦂🕸🕷🦗🦟🦕🐙🦑🦐🦞🦀🐡🐠🐟🦓🐆🐅🐊🦭🦈🐋🐳🐬🦍🦧🦣🐘🦛🦏🐪🐫🦒🐑🐏🐖🐎🐄🐂🐃🦬🦘🦙🐐🦌🐕🐩🦮🐕‍🦺🐈🐈‍⬛🕊🦩🦢🦜🦚🦤🦃🐓🪶🐇🦝🦨🦡🦫🦦🦥🐁🐀🌳🌲🎄🌵🐲🐉🐾🦔🐿🌴🪵🌱🌿☘️🍀🎍🪴🎋🪨🪸🐚🍄🪹🪺🍁🍂🍃🌾💐🌷🌹🥀🪷🌺🌸🌼🌗🌖🌕🌚🌜🌛🌝🌞🌻🌘🌑🌒🌓🌔🌙🌎🌍🌏🔥💥☄️⚡️✨🌟⭐️💫🪐🌪🌈☀️🌤⛅️🌥☁️🌦🌧💧💨🌬⛄️☃️❄️🌨🌩⛈💦🫧☔️☂️🌊🍏🍎🍐🍊🍋🍌🍉🍇🍓🍅🥝🥥🍍🥭🍑🍒🍈🫐🍆🥑🥦🥬🥒🌶🫑🌽🥕🥖🍞🥯🥐🍠🥔🧅🧄🫒🥨🧀🥚🍳🧈🥞🧇🥓🥩🥪🫓🍕🍟🍔🌭🦴🍖🍗🥙🧆🌮🌯🫔🥗🥘🫕🥫🦪🥟🍱🍣🍛🍲🍜🍝🫙🍤🍙🍚🍘🍥🥠🥮🍢🍡🍭🍮🎂🍰🧁🥧🍦🍨🍧🍬🍫🍿🍩🍪🌰🥜🫘🍯🧋🥤🧃🍵☕️🫖🍼🫗🥛🍶🍺🍻🥂🍷🥃🍸🍹🧉🧂🥢🥡🥣🍽🍴🥄🧊🍾⚽️🏀🏈⚾️🥎🎾🏐🏉🥏🪃🏏🥍🏑🏒🏸🏓🪀🎱🥅⛳️🪁🛝🏹🎣🤿🥊🥋🏂⛷🎿🥌⛸🛷🛼🛹🎽🪂🏋️‍♀️🏋️🏋️‍♂️🤼‍♀️🤼🤼‍♂️🤸‍♀️🤸🏌️‍♀️🤾‍♂️🤾🤾‍♀️🤺⛹️‍♂️⛹️⛹️‍♀️🤸‍♂️🏌️🏌️‍♂️🏇🧘‍♀️🧘🧘‍♂️🏄‍♀️🏄🏄‍♂️🚣‍♂️🚣🚣‍♀️🤽‍♂️🤽🤽‍♀️🏊‍♂️🏊🏊‍♀️🧗‍♀️🧗🧗‍♂️🚵‍♀️🚵🚵‍♂️🚴‍♀️🚴🚴‍♂️🎫🎗🏵🎖🏅🥉🥈🥇🏆🎟🎪🤹‍♀️🤹🤹‍♂️🎭🩰🎨🎬🪗🎺🎷🪘🥁🎹🎼🎧🎤🎸🪕🎻🎲♟🎯🎳🎮🎰🧩🚗🚕🚙🚌🚎🏎🚓🚑🚒🩼🦼🦽🦯🚜🚛🚚🛻🚐🛴🚲🛵🏍🛺🛞🚨🚔🚍🚝🚞🚋🚃🚟🚠🚡🚖🚘🚄🚅🚈🚂🚆🚇🚊🚉✈️🛶🚁🛸🚀🛰💺🛩🛬🛫⛵️🚤🛥🛳🚢🛟⛴⚓️🪝🗼🗽🗿🗺🚏🚥🚦🚧⛽️🏰🏯🏟🎡🎢🎠⛲️⛱🏖🛖⛺️🏕🗻🏔⛰🌋🏜🏝🏠🏡🏘🏚🏗🏭🏢🏬🏣🏛💒🏩🏫🏪🏨🏦🏥🏤⛪️🕌🕍🛕🕋⛩🛤🛣🗾🌆🌇🎆🎇🌠🌄🌅🏞🎑🏙🌃🌌🌉🌁⌚️📱📲💻⌨️🖥🖨🖱🖲📸📷📼📀💿💾💽🗜🕹📹🎥📽🎞📞☎️📟📠📺🕰⏰⏲⏱🧭🎛🎚🎙📻⌛️⏳📡🔋🪫🔌💡🔦🕯🪙💷💶💴💵💸🛢🧯🪔💰💳🪪💎⚖️🪜🧰🪛🔧🧱🪤⚙️🔩🪚⛏🛠⚒🔨⛓🧲🔫💣🧨🪓🔪🗡⚔️🧿📿🔮🏺⚱️🪦⚰️🚬🛡🪬💈⚗️🔭🔬🕳🩻🩹🩺🧹🌡🧪🧫🦠🧬🩸💉💊🪠🧺🧻🚽🚰🚿🛁🛀🧼🚪🗝🔑🛎🧴🪣🧽🪒🪥🪑🛋🛏🛌🧸🪆🖼🪞🪟🎊🪅🪄🎀🎏🎈🎁🛒🛍🎉🎎🏮🎐🪩🧧✉️📩📨📫📪🪧🏷📦📤📥💌📧📬📭📮📯📜📃📄📑🧾📇🗑📅📆🗓🗒📉📈📊🗃🗳🗄📋📂📁🗂🗞📰📖📚📙📘📗📕📒📔📓🔖🧷🔗📎🖇📐📏🧮📌✏️📝🖍🖌✒️🖋🖊✂️📍🔍🔎🔏🔐🔒🔓❤️🧡💛💚💙💜🤎🖤🤍⚪️⚫️🟤🟣🔵🟢🟡🟠🔴🟥🟧🟨🟩🟦🟫⬛️⬜️❤️‍🩹❣️❤️‍🔥💔🏿🏾🏽🏼🏻💕💞💓💗💖💘💝💟☮️☦️☯️🕎🔯✡️☸️🕉☪️✝️🛐⛎♈️♉️♊️♋️♌️♍️♎️⚛️🆔♓️♒️♑️♐️♏️🉑☢️📴📳🈶🈚️🈸🈺🈷️✴️🈲🈹🈵🈴㊗️㊙️🉐💮🆚🅰️🅱️🆎🅾️🆑🆘❌⭕️🛑🚳🚯🚷♨️💢💯🚫📛⛔️🚱🔞📵🚭❗️❕❓❔‼️🔰⚜️🔱🚸⚠️〽️🔆🔅⁉️♻️✅🈯️💹❇️✳️❎🌐💠🈳🛗🅿️♿️🚾🏧💤🌀Ⓜ️🈂️🛂🛃🛄🛅🚹🚺🚼⚧🔡🔤ℹ️🔣🈁📶🎦🚮🚻🔠🆖🆗🆙🆒🆕🆓0️⃣1️⃣🔟9️⃣8️⃣7️⃣6️⃣5️⃣4️⃣3️⃣2️⃣🔢#️⃣*️⃣⏏️▶️⏸⏯⏹⏺🔽🔼◀️⏬⏫⏪⏩⏮⏭➡️⬅️⬆️⬇️↗️↘️↙️↖️↕️🔄🔂🔁🔀⤵️⤴️↩️↪️↔️🔃🎵🎶➕➖➗✖️🟰♾️🔛🔙🔚👁‍🗨®️©️™️💱💲🔝🔜〰️➰➿✔️☑️🔘🔺▫️▪️🔲🔳🔷🔶🔹🔸🔻◾️◽️◼️◻️🔈🔇🔉🔊🔔♥️♣️♠️🗯💭💬📢📣🔕♦️🃏🎴🀄️🕐🕑🕒🕓🕔🕝🕜🕛🕚🕙🕘🕗🕖🕕🕞🕟🕠🕡🕢🕣🕤🕥🕦🕧🏳️🏴🏴‍☠️🏁🚩🏳️‍🌈🏳️‍⚧️🇺🇳🇮🇸🇦🇱🇦🇼🇦🇷🇩🇿🇦🇪🇺🇸🇦🇫🇦🇿🇮🇪🇦🇲🇦🇮🇦🇴🇦🇬🇦🇩🇾🇪🇬🇧🏴󠁧󠁢󠁥󠁮󠁧󠁿🏴󠁧󠁢󠁳󠁣󠁴󠁿🇺🇬🇼🇫🇮🇩🇮🇳🇮🇷🇮🇶🇮🇹🇮🇱🏴󠁧󠁢󠁷󠁬󠁳󠁿🇺🇦🇺🇿🇺🇾🇪🇨🇪🇬🇪🇪🇸🇿🇪🇹🇪🇷🇨🇻🇬🇭🇧🇶🇳🇱🇴🇲🇦🇽🇦🇹🇦🇺🇸🇻🇬🇬🇬🇾🇰🇿🇨🇦🇮🇨🇬🇦🇨🇲🇬🇲🇰🇬🇰🇮🇬🇷🇨🇼🇨🇺🇨🇾🇬🇼🇬🇳🇰🇭🇬🇹🇬🇵🇬🇺🇰🇼🇨🇰🇬🇱🇨🇽🇬🇩🇭🇷🇨🇬🇨🇴🇰🇲🇽🇰🇨🇷🇨🇨🇨🇮🇰🇪🇰🇾🇨🇩🇸🇦🇬🇸🇼🇸🇧🇱🇸🇹🇿🇲🇵🇲🇸🇲🇸🇽🇸🇬🇸🇾🇬🇪🇯🇲🇯🇪🇬🇮🇩🇯🇸🇱🇿🇼🇨🇭🇸🇪🇸🇩🇪🇸🇸🇷🇱🇰🇸🇰🇸🇮🇸🇧🇸🇴🇱🇨🇸🇭🇻🇨🇰🇳🇷🇸🇸🇳🇸🇨🇹🇨🇹🇭🇹🇯🇨🇿🇮🇴🇹🇩🇹🇳🇨🇱🇹🇲🇹🇹🇩🇲🇩🇴🇹🇰🇹🇬🇩🇪🇩🇰🇹🇻🇹🇷🇹🇴🇳🇬🇳🇷🇳🇦🇳🇺🇳🇮🇳🇪🇳🇨🇻🇦🇵🇰🇭🇹🇧🇭🇳🇴🇳🇫🇳🇵🇳🇿🇻🇺🇧🇸🇵🇬🇧🇲🇵🇼🇵🇾🇧🇧🇵🇸🇭🇺🇫🇰🇫🇴🇵🇷🇧🇹🇫🇮🇵🇭🇫🇯🇵🇳🇧🇩🇧🇷🇫🇷🇧🇬🇧🇫🇧🇳🇧🇮🇻🇳🇧🇯🇻🇪🇵🇹🇧🇴🇧🇼🇧🇦🇵🇱🇧🇪🇵🇪🇧🇿🇧🇾🇭🇳🇲🇭🇲🇴🇲🇬🇾🇹🇲🇼🇲🇱🇲🇹🇲🇶🇲🇨🇲🇿🇲🇷🇲🇺🇲🇽🇲🇲🇫🇲🇮🇲🇲🇾🇲🇻🇲🇩🇲🇦🇲🇳🇲🇪🇲🇸🇯🇴🇱🇻🇱🇧🇱🇸🇷🇼🇱🇺🇷🇴🇱🇷🇱🇮🇱🇾🇱🇹🇷🇪🇷🇺🇻🇬🇪🇺🇰🇷🇪🇭🇭🇰🇬🇶🇨🇫🇨🇳🇹🇱🇸🇸🇦🇶🇯🇵🎌🇬🇫🇵🇫🇰🇵🇲🇵🇲🇰🇦🇸🇻🇮🇹🇫✡✳✴❇☯✂☘✉✔✘⚧✏✒♟" + \
"]+)"

LABEL_EQUIVALENCE = pd.read_excel(googleSheetURL, sheet_name="Fact Check Equivalence", header=None).to_numpy().tolist()

def processPost(str, swapII):
  str = removeHebrewAndArabic(str)

  if(swapII):
    str = swapIdentifyingInfo(str)

  return str

def removeHebrewAndArabic(str):
  str = str.translate( { ord(i): None for i in HEBREW_CHARACTERS } )
  str = str.translate( { ord(i): None for i in ARABIC_CHARACTERS } )
  return str

def swapIdentifyingInfo(str):
  splitStr = re.split(SPLIT_DELIMS, str)
  for i in range(0, len(splitStr)):
    replacement = None
    if(splitStr[i].lower() in ISRAEL_IDENTIFIERS):
      replacement = PALESTINE_IDENTIFIERS[ISRAEL_IDENTIFIERS.index(splitStr[i].lower())]
    elif(splitStr[i].lower() in PALESTINE_IDENTIFIERS):
      replacement = ISRAEL_IDENTIFIERS[PALESTINE_IDENTIFIERS.index(splitStr[i].lower())]

    if(replacement != None):
      if(splitStr[i] == splitStr[i].capitalize()):
          splitStr[i] = replacement.capitalize()
      elif(splitStr[i].isupper()):
          splitStr[i] = replacement.upper()
      else:
          splitStr[i] = replacement

  return "".join(splitStr)


def standardizeLabel(str):
  for i in range(0, len(LABEL_EQUIVALENCE)):
    if(str in LABEL_EQUIVALENCE[i]):
      return LABEL_EQUIVALENCE[i][0]
  assert("LABEL NOT FOUND")
  return str.capitalize()

PROCESSED_DATA = []

for i in range(0, len(IMPORTED_DATA)):
  newEntry = [processPost(IMPORTED_DATA[i][0], i % 2), # swap identifying info for every other entry
              standardizeLabel(IMPORTED_DATA[i][1])]
  PROCESSED_DATA.append(newEntry)

print(np.array(PROCESSED_DATA))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight

from imblearn.over_sampling import SMOTE

# Suppressing ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

text_data = [entry[0] for entry in PROCESSED_DATA]
labels = [entry[1] for entry in PROCESSED_DATA]

for i in range(len(labels)):
  if labels[i] == 'True':
    labels[i] = True
  if labels[i] == 'False':
    labels[i] = False

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)

# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42, stratify=labels)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

mlp = MLPClassifier(hidden_layer_sizes=(100,), alpha=0.001, learning_rate_init=0.01, max_iter=1000, random_state=42, momentum=0.9, solver='adam')
mlp.fit(X_train_res, y_train_res)


# Evaluate this model
predict_test = mlp.predict(X_test)
predict_train = mlp.predict(X_train_res)


# Evaluate the best model on the test data
print("Test Classification Report:\n", classification_report(y_test, predict_test))
print("Train Classification Report:\n", classification_report(y_train_res, predict_train))


Test Classification Report:
               precision    recall  f1-score   support

       False       0.62      0.71      0.67         7
        True       0.33      0.25      0.29         4

    accuracy                           0.55        11
   macro avg       0.48      0.48      0.48        11
weighted avg       0.52      0.55      0.53        11

Train Classification Report:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00        28
        True       1.00      1.00      1.00        28

    accuracy                           1.00        56
   macro avg       1.00      1.00      1.00        56
weighted avg       1.00      1.00      1.00        56



In [ ]:
# Bayes - Zaid

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

text_data = [entry[0] for entry in PROCESSED_DATA]
labels = [entry[1] for entry in PROCESSED_DATA]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

best_alpha = 0
best_accuracy = 0
best_y_pred = None

for alpha in np.arange(0.1, 1.1, 0.1):
    model = MultinomialNB(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Alpha: {alpha:.1f}, Accuracy: {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_alpha = alpha
        best_y_pred = y_pred

print(f"Best Alpha: {best_alpha}, Best Accuracy: {best_accuracy}")
print("Confusion Matrix for Best Alpha:\n", confusion_matrix(y_test, best_y_pred))
print("Classification Report for Best Alpha:\n", classification_report(y_test, best_y_pred))


Alpha: 0.1, Accuracy: 0.5455
Alpha: 0.2, Accuracy: 0.6364
Alpha: 0.3, Accuracy: 0.7273
Alpha: 0.4, Accuracy: 0.8182
Alpha: 0.5, Accuracy: 0.9091
Alpha: 0.6, Accuracy: 0.9091
Alpha: 0.7, Accuracy: 0.9091
Alpha: 0.8, Accuracy: 0.9091
Alpha: 0.9, Accuracy: 0.9091
Alpha: 1.0, Accuracy: 0.8182
Best Alpha: 0.5, Best Accuracy: 0.9090909090909091
Confusion Matrix for Best Alpha:
 [[9 0]
 [1 1]]
Classification Report for Best Alpha:
               precision    recall  f1-score   support

       False       0.90      1.00      0.95         9
        True       1.00      0.50      0.67         2

    accuracy                           0.91        11
   macro avg       0.95      0.75      0.81        11
weighted avg       0.92      0.91      0.90        11



In [ ]:
# SVM - Ahyush
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

data = np.array(PROCESSED_DATA)
X = data[:, 0]
y = data[:, 1]
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.25, random_state=42)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['rbf', 'linear','poly']
}
svm = SVC(class_weight='balanced')
grid_search = GridSearchCV(svm, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_smote, y_train_smote)

y_pred = grid_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))
print("Best Parameters:", grid_search.best_params_)



print(y_train_smote)

print("-------------------------------")

print(y_test)

              precision    recall  f1-score   support

       False       0.75      0.90      0.82        10
        True       0.50      0.25      0.33         4

    accuracy                           0.71        14
   macro avg       0.62      0.57      0.58        14
weighted avg       0.68      0.71      0.68        14

Best Parameters: {'C': 1, 'gamma': 1, 'kernel': 'linear'}
['False' 'False' 'False' 'True' 'True' 'False' 'True' 'True' 'False'
 'True' 'False' 'False' 'False' 'False' 'False' 'True' 'True' 'True'
 'True' 'False' 'False' 'False' 'False' 'False' 'False' 'True' 'False'
 'False' 'False' 'True' 'True' 'False' 'True' 'True' 'False' 'False'
 'False' 'True' 'False' 'False' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True']
-------------------------------
['False' 'False' 'False' 'False' 'False' 'True' 'False' 'False' 'True'
 'False' 'False' 'False' 'True' 'True']
